# Broadcast

### But : Faire un broadcast, départ décalé 


#### Informations générales

In [1]:
experiment_name = "Time_Bcast"

In [2]:
%%bash -s
git log -n 1
python --version

commit 8c4809674eb7197f2589a99e3def6282ca769f40
Merge: d9c55a6 359473e
Author: Martin Router Queen <jaiunpetitcerveau@gmail.com>
Date:   Thu May 31 15:39:25 2018 +0200

    Merge branch 'master' of github.com:hoobaeboo/Stage-POLARIS


Python 2.7.13


#### Préparation de l'environnement

In [3]:
%%bash -s
pip3 install --user execo
pip3 install --user requests

  Using cached https://files.pythonhosted.org/packages/49/df/50aa1999ab9bde74656c2919d9c0c085fd2b3775fd3eca826012bef76d8c/requests-2.18.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/27/cc/6dd9a3869f15c2edfab863b992838277279ce92663d334df9ecf5106f5c6/idna-2.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/e6/92ad559b7192d846975fc916b65f667c7b8c3a32bea7372340bfe9a15fa5/certifi-2018.4.16-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/cb/6965947c13a94236f6d4b8223e21beb4d576dc72e8130bd7880f600839b8/urllib3-1.22-py2.py3-none-any.whl


In [4]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import datetime
import re
from shutil import copy
from subprocess import check_output
import random
from threading import Thread

##### Réservation noeuds

In [5]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 4
# Temps réservation
walltime = "3:0:0" 

# filters out Nantes's econome cluster
resources_selection = "-p \"cluster in ('ecotype','parasilo','grisou')\"" 

# Nancy
site = "rennes"

In [69]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 1014900


On les range

In [70]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('parasilo-5.rennes.grid5000.fr'),
 Host('parasilo-6.rennes.grid5000.fr'),
 Host('parasilo-7.rennes.grid5000.fr'),
 Host('parasilo-8.rennes.grid5000.fr')]

##### Déploiement de l'environnement

In [71]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = 'src/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [72]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

rennes: Deployment #D-474dbfd5-0b01-452a-9e5b-8b45dcba2cdd started
rennes: Grab the key file /home/nezzine/.ssh/authorized_keys
rennes: Grab the tarball file http://public.nancy.grid5000.fr/~pneyron/debian9-x64-bigdata-tutorial.tar.gz
rennes: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgz
rennes: Launching a deployment on parasilo-[5-8].rennes.grid5000.fr
rennes: Performing a Deploy[SetDeploymentEnvUntrusted] step
rennes:   switch_pxe
rennes:   reboot
rennes:    * Performing a soft reboot on parasilo-[5-8].rennes.grid5000.fr
rennes:   wait_reboot
rennes:   send_key_in_deploy_env
rennes:   create_partition_table
rennes:   format_deploy_part
rennes:   mount_deploy_part
rennes:   format_tmp_part
rennes:   format_swap_part
rennes: End of step Deploy[SetDeploymentEnvUntrusted] after 205s
rennes: Performing a Deploy[BroadcastEnvKascade] step
rennes:   send_environment
rennes:    * Broadcast time: 70s
rennes:   manage_admin_post_install
rennes:   manage_user_pos

rennes: Warning: Permanently added 'rennes.grid5000.fr,172.16.111.106' (ECDSA) to the list of known hosts.
rennes: Connection to rennes.grid5000.fr closed.


#### Test : commande simple 'ls'

In [ ]:
# execo.action.Remote(cmd,hosts,connection_params,process_args)
Remote_test = execo.action.Remote(cmd='ls',hosts=nodes,connection_params=None, process_args=None)
Remote_test.run().ok

#### Installation environnement sur noeud

In [73]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [74]:
Remote_install.run().ok

True

#### Version Simgrid

In [21]:
execo.action.Remote(cmd='touch smpi_version.txt && smpicc --version > smpi_version.txt'
                    , hosts=nodes, connection_params={'user': 'root'
                    }).run()
version = ''
for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i], remote_files=['~/smpi_version.txt'
                     ], local_location='./smpicc_version-'
                     + get_host_shortname(nodes[i]) + '.txt',
                     connection_params={'user': 'root'}).run()
    version = version + execo.Process(cmd='head -n 1 ./smpicc_version-'
            + get_host_shortname(nodes[i]) + '.txt'
            ).run().stdout.replace('\n', '  ')
    execo.Process(cmd='rm ./smpicc_version-'
                  + get_host_shortname(nodes[i]) + '.txt').run()
version


'SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  SimGrid version 3.18  '

In [75]:
path_base = "~/Stage-POLARIS/SimGrid/examples/SMPI/"+experiment_name
path_src = path_base + '/src/'
path_rslt = path_base + '/results/'

#### Récupération fichiers sources

In [ ]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',hosts=nodes,connection_params={'user':'root'})
Src_get.run().ok

In [ ]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path_src+"griffon.xml",path_src+"griffon_hostfile.txt",path_src+"broadcast.c",path_src+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})
Remote_smpi_env.run().ok

#### Execution de l'expérience

In [ ]:
Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10 N=2 DELAY=\"0 0\"',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

#### Récupération des résultats

In [ ]:
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))
Result_get.run().ok

In [ ]:
for i in range(0, nodecount):
    execo.action.Get(hosts = nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location = path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

#### Delayer

In [76]:
# Parsing the results
def parser(num_node,date_precise):
    with open('/home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/'
               + date_precise.split('_')[0] + '/'
              + get_host_shortname(nodes[num_node]) + '_'
              + date_precise
              + '.txt') as f:
        return f.readlines()

In [77]:
# Converting and organizing data
def tab_conv(l,NPROCS) :
    S = init_tab(NPROCS, None)
        
    # Taking useful data    
    for i in range(0,NPROCS) :
        S[int(l[i].split()[1])] = int(float(l[i].split()[5]))
    return S

In [78]:
def delayer(num_node,NPROCS,date):
    D = init_tab(NPROCS,0)
    
    raw_data = parser(num_node,date)
    
    S = tab_conv(raw_data,NPROCS)
    
    for p in range(0, NPROCS):
        D[p] = S[p] - min(S)
    return D

In [55]:
# Example
delayer(0,10,'2018-06-04_12:38')

[9, 4, 4, 4, 8, 0, 6, 2, 7, 7]

In [79]:
def init_tab(num, elt) :
    D = [elt] * num
    return D

##### Shortcut

In [17]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
Remote_smpi_env = execo.action.Put(hosts=nodes, local_files=[path_src
                                   + 'griffon.xml', path_src
                                   + 'griffon_hostfile.txt', path_src
                                   + 'broadcast.c', path_src + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))
def parl_exec () :
    Result_get.run().ok
    # Nbr pcs
    NPROCS = 10
    
    # Nbr itérations
    N = 100
    
    D = init_tab(NPROCS,0)
    delay = ' '.join(str(e) for e in D)
    date = datetime.datetime.now().strftime('%Y-%m-%d');
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
    print(date_precise)
    
    # TODO //
    for i in range (0,nodecount) :
        MIN = (int)((i*N)/len(nodes))
        MAX = (int)(((i+1)*N)/len(nodes))
        print(get_host_shortname(nodes[i])+" => "+str(i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(MIN)
             +" MAX = "+str(MAX)
             +" N = "+str(N)
             +" DELAY = '"+ delay +"'"
            )
        bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(MIN) 
                                  +' MAX='+str(MAX)+' N='+str(N)
                                  +" DELAY='"+delay
                                  +"'",hosts=nodes[i],connection_params={'user':'root'}).run().ok
        print(bool)
        execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok
        print(delayer(i,NPROCS,date_precise))
    return;

# Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS=2 MIN=0 MAX=10',hosts=nodes,connection_params={'user':'root'})



Src_get.run().ok
Remote_smpi_env.run().ok

# Remote_smpi.run().ok

parl_exec()

for i in range(0, nodecount):
    execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location=path_rslt
                     + datetime.datetime.now().strftime('%Y-%m-%d')
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M'
                     ) + '.txt', connection_params={'user': 'root'
                     }).run().ok

2018-06-04 11:09:39,347 WARNING: terminated: <SshProcess('mkdir -p experiments_src', Host('parasilo-3.rennes.grid5000.fr'), connection_params={'user': 'root'}, name=mkdir -p experiments_src, started=True, start_date=2018-06-04 11:09:39+02:00, ended=True, end_date=2018-06-04 11:09:39+02:00, killed=False, error=False, error_reason=None, timeouted=False, expect_fail=False, write_error=False, exit_code=65280, ok=False, pid=17327, real cmd=('ssh', '-tt', '-o', 'BatchMode=yes', '-o', 'PasswordAuthentication=no', '-o', 'StrictHostKeyChecking=no', '-o', 'UserKnownHostsFile=/dev/null', '-o', 'ConnectTimeout=20', '-o', 'User=root', 'parasilo-3.rennes.grid5000.fr', 'mkdir -p experiments_src'))>
stdout:

stderr:
Permission denied (publickey,password).

2018-06-04 11:09:39,355 WARNING: terminated: <SshProcess('mkdir -p experiments_src', Host('parasilo-28.rennes.grid5000.fr'), connection_params={'user': 'root'}, name=mkdir -p experiments_src, started=True, start_date=2018-06-04 11:09:39+02:00, ended

False
2018-06-04 11:09:40,404 WARNING: terminated: <Process('scp -o BatchMode=yes -o PasswordAuthentication=no -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -o ConnectTimeout=20 -rp -o User=root parasilo-28.rennes.grid5000.fr:~/experiments_src/bcast_results.txt ~/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/2018-06-04/parasilo-28_2018-06-04_11:09.txt', shell=True, name=scp -o BatchMode=yes -o PasswordAuth..., started=True, start_date=2018-06-04 11:09:40+02:00, ended=True, end_date=2018-06-04 11:09:40+02:00, killed=False, error=False, error_reason=None, timeouted=False, expect_fail=False, write_error=False, exit_code=256, ok=False, pid=17360)>
stdout:

stderr:
Permission denied (publickey,password).



FileNotFoundError: [Errno 2] No such file or directory: '/home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/Time_Bcast/results/2018-06-04/parasilo-28_2018-06-04_11:09.txt'

#### //

In [107]:
Src_get = execo.action.Remote(cmd='mkdir -p experiments_src',
                              hosts=nodes,
                              connection_params={'user': 'root'})
Remote_smpi_env = execo.action.Put(hosts=nodes, local_files=[path_src
                                   + 'griffon.xml', path_src
                                   + 'griffon_hostfile.txt', path_src
                                   + 'broadcast.c', path_src + 'Makefile'],
                                   remote_location='~/experiments_src',
                                   connection_params={'user': 'root'})
Result_get = \
    execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/SimGrid/examples/SMPI/'
                   + experiment_name + '/results/'
                  + datetime.datetime.now().strftime('%Y-%m-%d'))

# On récup les src
Src_get.run().ok
Remote_smpi_env.run().ok


True

In [108]:
class ParSimer(Thread):

    """Thread chargé simplement d'afficher une lettre dans la console."""

    def __init__(self,i,MIN,MAX,N,DELAY):
        Thread.__init__(self)
        self.MIN = MIN
        self.MAX = MAX
        self.N = N
        self.DELAY = DELAY
        self.i = i
        
    def set_delay(self,delay):
        self.DELAY = delay
    
    def get_delay(self):
        return self.DELAY
    
    def run(self):
        """Code à exécuter pendant l'exécution du thread."""
        print(get_host_shortname(nodes[self.i])+" => "+str(self.i)+" NPROCS = "+str(NPROCS)
             +" MIN = "+str(self.MIN)
             +" MAX = "+str(self.MAX)
             +" N = "+str(self.N)
             +" DELAY = '"+ self.DELAY +"'\n"
            )
        bool = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run NPROCS='
                                  +str(NPROCS)+' MIN='+str(self.MIN) 
                                  +' MAX='+str(self.MAX)+' N='+str(self.N)
                                  +" DELAY='"+self.DELAY
                                  +"'",hosts=nodes[self.i],connection_params={'user':'root'}).run().ok


In [109]:
# Test trajectoire raccrochée à la trajectoire initiale
def coherence(tab):
     return any(x == True for x in tab)

In [110]:
# SIMULATION
## Simulation simple en diminuant le nbr de machines 
def simulation(N,NPROCS):
    # Tableau de threads
    machines = init_tab(nodecount,None)
    
    # D initial
    D = init_tab(NPROCS,0)
    delay = ' '.join(str(e) for e in D)
    
    # Dates
    date = datetime.datetime.now().strftime('%Y-%m-%d');
    date_precise = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')

    # Test interprocessus
    join = init_tab(nodecount-1,None)
    
    Result_get.run().ok
    
    for j in range(0,nodecount) :
        print("Round "+str(j) +"\n")
        # Paramètrage des machines
        for i in range (0,nodecount) :           
            MIN = (int)((i*N)/nodecount)
            MAX = (int)(((i+1)*N)/nodecount)
            machines[i] = ParSimer(i,MIN,MAX,N,delay)
            # Vérification cohérence & calcul des new D.
            if (i != 0 and j!=0 ):
                S = delayer(i-1,NPROCS,date_precise)
                join[i-1] = all(x1 == x2 for x1, x2 in zip(S, machines[i].get_delay()))
                machines[i].set_delay(' '.join(str(e) for e in S))  
            # Affichage de cohérence    
            if(i == nodecount-1):
                print(str(join)+"\n")
            if(coherence(join)):
                break;
                          
        # Lancement de la simulation
        for i in range(j,nodecount):
            machines[i].start()
        # Fin de simulation
        for i in range(j,nodecount) :
            machines[i].join()
            # Récupération des résultats
            execo.action.Get(hosts=nodes[i],
                     remote_files=['~/experiments_src/bcast_results.txt'
                     ],
                     local_location= path_rslt
                     + date
                     + '/' + get_host_shortname(nodes[i]) + '_'
                     + date_precise + '.txt', connection_params={'user': 'root'
                     }).run().ok  

In [111]:
# Nbr pcs
NPROCS = 10
    
# Nbr itérations
N = 100

simulation(N,NPROCS)

Round 0

[None, None, None]

parasilo-5 => 0 NPROCS = 10 MIN = 0 MAX = 25 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'

parasilo-6 => 1 NPROCS = 10 MIN = 25 MAX = 50 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'

parasilo-7 => 2 NPROCS = 10 MIN = 50 MAX = 75 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'

parasilo-8 => 3 NPROCS = 10 MIN = 75 MAX = 100 N = 100 DELAY = '0 0 0 0 0 0 0 0 0 0'

Round 1

[False, False, False]

parasilo-6 => 1 NPROCS = 10 MIN = 25 MAX = 50 N = 100 DELAY = '4 5 4 4 7 10 7 1 7 0'

parasilo-7 => 2 NPROCS = 10 MIN = 50 MAX = 75 N = 100 DELAY = '2 6 6 3 7 0 1 0 2 1'

parasilo-8 => 3 NPROCS = 10 MIN = 75 MAX = 100 N = 100 DELAY = '7 3 3 7 0 6 7 9 2 5'

Round 2

[False, False, False]

parasilo-7 => 2 NPROCS = 10 MIN = 50 MAX = 75 N = 100 DELAY = '4000 5003 4002 4002 6999 10006 7006 998 7000 0'

parasilo-8 => 3 NPROCS = 10 MIN = 75 MAX = 100 N = 100 DELAY = '2004 7008 6003 3003 7003 0 1002 2 3006 1005'

Round 3

[False, False, False]

parasilo-8 => 3 NPROCS = 10 MIN = 75 MAX = 100 N = 100 D

KeyboardInterrupt: 

#### Publication des résultats

In [112]:
%%bash -s $experiment_name $site
git add .
git commit -m 'Automatic results update : '"$1"'---'"$2"' Format'

[master 477c2d8] Automatic results update : Time_Bcast---rennes Format
 Committer: Najwa Ez Zine <nezzine@frennes>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 33 files changed, 531 insertions(+), 221 deletions(-)
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-04/parasilo-5_2018-06-04_15:28.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-04/parasilo-5_2018-06-04_15:32.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-04/parasilo-5_2018-06-04_15:41.txt
 create mode 100644 SimGrid/examples/SMPI/Time_Bcast/results/2018-06-04/parasilo-5_

#### Fin d'expérience : suppression du job

In [10]:
oardel(jobid)

TypeError: 'int' object is not iterable